<a href="https://colab.research.google.com/github/MarinaSupiot/projet_7/blob/main/Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 533.5/533.5 kB 5.5 MB/s eta 0:00:00


In [ ]:
!pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 85.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.6 MB/s eta 0:00:00
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 7.0.0
    Uninstalling importlib-metadata-7.0.0:
      Successfully uninstalled importlib-metadata-7.0.0


In [ ]:
! pip install streamlit-shap

In [203]:
%%writefile app.py
import streamlit as st
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import plotly.express as px
import plotly.graph_objects as go
import shap
import lightgbm
from streamlit_shap import st_shap
import base64





# Создание двух колонок для размещения текста и изображения
col1, col2 = st.columns([2, 10])

# В первой колонке разместите текст
col2.markdown('<p style="font-size:36px; font-weight:bold;">Bienvenue dans l\'application<br>Prêt à dépenser!</p>', unsafe_allow_html=True)

# Во второй колонке разместите изображение
col1.image('/content/drive/MyDrive/Projet 7/Prets_a_depenser.png', width=110)




@st.cache_resource
def load_data_and_model():
    # Считывание данных
    df_test = pd.read_csv("/content/drive/MyDrive/Projet 7/test_preprocess_reduit.csv")

    # Загрузка модели
    with open('/content/drive/MyDrive/Projet 7/model_su04.pkl', 'rb') as file:
        model = pickle.load(file)

    return df_test, model

# Загрузка данных и модели
df_test, model = load_data_and_model()

# Выбор клиента
option = st.sidebar.selectbox(
   "Entrer le numero de client",
   (df_test['SK_ID_CURR'].unique()),
   index=None,
   placeholder="Select le numero de client",
)



# Добавление кнопки "Statut de la demande" в боковую панель
if st.sidebar.button("Statut de la demande"):
    # Здесь вы можете добавить код для обработки нажатия кнопки

    # Открытие модели
    #with open('/content/drive/MyDrive/Projet 7/model_su04.pkl', 'rb') as file:
        #model = pickle.load(file)

    # Фильтрация данных для выбранного клиента
    filtered_df = df_test[df_test['SK_ID_CURR'] == option]

    # Проверка наличия данных для выбранного клиента
    if filtered_df.empty:
        probability_value = None
    else:
        # Преобразование входных данных в 2D массив
        input_data = filtered_df.drop(columns=['SK_ID_CURR']).values

        # Проверка, что данные не пусты и имеют два измерения
        if len(input_data.shape) == 2 and input_data.shape[0] > 0 and input_data.shape[1] > 0:
            # Прогнозирование вероятности
            prediction_probabilities = model.predict_proba(input_data)
            # Применение порога
            prediction = (prediction_probabilities[:, 1] > 0.556).astype(int)

            if prediction == 1:
                st.markdown('<p style="font-size:40px; text-align:center; color:#00008B	; font-weight: bold; ">Désolé, votre demande a été rejetée.</p>', unsafe_allow_html=True)


            else:
                st.markdown('<p style="font-size:40px; text-align:center; color:#00008B	; font-weight: bold; ">Félicitation ! Votre demande est acceptée !</p>', unsafe_allow_html=True)
                st.balloons()

            # Получите значение вероятности
            probability_value = prediction_probabilities[0][1]
        else:
            st.write("Erreur : Les données d'entrée sont vides ou n'ont pas deux dimensions.")

    # Определите цвета для красной и зеленой областей
    red_color = '#FF0000'
    green_color = '#00FF00'

    # Определите границы для красной и зеленой областей
    green_range = (0, 0.556)
    red_range = (0.556, 1)

    # Создайте данные для графика
    fig = go.Figure()

    # Добавьте круглый Gauge, только если значение вероятности определено
    if probability_value is not None:
        fig.add_trace(go.Indicator(
            mode="gauge+number",
            value=probability_value,
            domain={'x': [0, 1], 'y': [0, 1]},
            gauge={
                'shape': "angular",
                'axis': {'range': [None, 1]},
                'bar': {'color': "darkblue"},
                'bgcolor': "white",
                'borderwidth': 2,
                'bordercolor': "gray",
                'steps': [
                    {'range': red_range, 'color': red_color},
                    {'range': green_range, 'color': green_color},
                ],
            }
        ))

        # Обновите макет графика
        fig.update_layout(
            autosize=False,
            margin=dict(l=20, r=20, b=20, t=40),
        )

        # Покажите график
        st.plotly_chart(fig)

option1 = st.sidebar.checkbox("Facteurs d'influence globaux")
option2 = st.sidebar.checkbox("Facteurs d'influence individuels")
option3 = st.sidebar.checkbox("Comparaison avec les autres clients")


@st.cache_data
def compute_shap_values(_explainer_shap, df):
    return _explainer_shap.shap_values(df)

# Проверка состояния чекбокса option1
if option1:
    explainer_shap = shap.TreeExplainer(model)
    df_test_indexed = df_test.set_index('SK_ID_CURR')

    # Проверка, что shap_values не является None
    shap_values = compute_shap_values(explainer_shap, df_test_indexed)
    if shap_values is not None:
        num_columns = st.slider("Sélectionnez le nombre de colonnes", min_value=1, max_value=80, value=10)

        # Вызов функции с кэшированием
        st_shap(shap.summary_plot(shap_values[1], df_test_indexed, show=False, max_display=num_columns))
        st.markdown(f'<img src="data:image/png;base64,{save_plot_to_base64(fig)}" alt="plot" style="width:100%;">', unsafe_allow_html=True)
    else:
        st.write("Erreur : Les valeurs SHAP ne sont pas disponibles.")


if option2:

  explainer_shap = shap.TreeExplainer(model)
  df_test_indexed = df_test.set_index('SK_ID_CURR')
  idx = df_test_indexed.index.get_loc(option)

    # Проверка, что shap_values не является None
  shap_values = compute_shap_values(explainer_shap, df_test_indexed)
    st_shap(shap.plots._waterfall.waterfall_legacy(explainer_shap.expected_value[0], shap_values[0][idx,:], val_sample_set_t.iloc[idx,:], max_display=15))

Overwriting app.py


In [206]:
%%writefile app.py
import streamlit as st
import pickle
import pandas as pd
import plotly.graph_objects as go
import shap
import numpy as np
import matplotlib as plt
from streamlit_shap import st_shap
from typing import Tuple, Any
import matplotlib.pyplot as plt_lib
import matplotlib.pyplot as plt

from io import BytesIO

def save_plot_to_base64(fig):
    image_stream = BytesIO()
    fig.savefig(image_stream, format='png')
    encoded_image = base64.b64encode(image_stream.getvalue()).decode()
    return encoded_image



# Функция для загрузки данных и модели
@st.cache_resource(show_spinner=False)
def load_data_and_model() -> Tuple[pd.DataFrame, Any]:
    df_test = pd.read_csv("/content/drive/MyDrive/Projet 7/test_preprocess_reduit.csv")
    with open('/content/drive/MyDrive/Projet 7/model_su04.pkl', 'rb') as file:
        model = pickle.load(file)
    return df_test, model


# Функция для вычисления shap_values
@st.cache_data(show_spinner=False)
def compute_shap_values(_explainer_shap, df):
    return _explainer_shap.shap_values(df)


# Загрузка данных и модели
df_test, model = load_data_and_model()

# Создание двух колонок для размещения текста и изображения
col1, col2 = st.columns([2, 10])

# Проверка, находится ли страница в начальном состоянии (при загрузке)
if "is_initial_load" not in st.session_state:
    # В первой колонке разместите текст только при загрузке страницы
    col2.markdown('<p style="font-size:36px; font-weight:bold;">Bienvenue dans l\'application<br>Prêt à dépenser!</p>', unsafe_allow_html=True)
    # Во второй колонке разместите изображение
    col1.image('/content/drive/MyDrive/Projet 7/Prets_a_depenser.png', width=110)
    st.session_state.is_initial_load = True




# Выбор клиента
option = st.sidebar.selectbox(
   "Entrer le numero de client",
   (df_test['SK_ID_CURR'].unique()),
   index=None,
   placeholder="Select le numero de client",
)

# Добавление кнопки "Statut de la demande" в боковую панель
if st.sidebar.button("Statut de la demande"):
    # Фильтрация данных для выбранного клиента
    filtered_df = df_test[df_test['SK_ID_CURR'] == option]

    # Проверка наличия данных для выбранного клиента
    if not filtered_df.empty:
        # Преобразование входных данных в 2D массив
        input_data = filtered_df.drop(columns=['SK_ID_CURR']).values

        # Проверка, что данные не пусты и имеют два измерения
        if len(input_data.shape) == 2 and input_data.shape[0] > 0 and input_data.shape[1] > 0:
            # Открытие модели
            # with open('/content/drive/MyDrive/Projet 7/model_su04.pkl', 'rb') as file:
            #    model = pickle.load(file)

            # Прогнозирование вероятности
            prediction_probabilities = model.predict_proba(input_data)
            # Применение порога
            prediction = (prediction_probabilities[:, 1] > 0.556).astype(int)

            if prediction == 1:
                st.markdown('<p style="font-size:40px; text-align:center; color:#00008B	; font-weight: bold; ">Désolé, votre demande a été rejetée.</p>', unsafe_allow_html=True)
            else:
                st.markdown('<p style="font-size:40px; text-align:center; color:#00008B	; font-weight: bold; ">Félicitation ! Votre demande est acceptée !</p>', unsafe_allow_html=True)
                st.balloons()

            # Получите значение вероятности
            probability_value = prediction_probabilities[0][1]

            # Определите цвета для красной и зеленой областей
            red_color = '#FF0000'
            green_color = '#00FF00'

            # Определите границы для красной и зеленой областей
            green_range = (0, 0.556)
            red_range = (0.556, 1)

            # Создайте данные для графика
            fig = go.Figure()

            # Добавьте круглый Gauge, только если значение вероятности определено
            fig.add_trace(go.Indicator(
                mode="gauge+number",
                value=probability_value,
                domain={'x': [0, 1], 'y': [0, 1]},
                gauge={
                    'shape': "angular",
                    'axis': {'range': [None, 1]},
                    'bar': {'color': "darkblue"},
                    'bgcolor': "white",
                    'borderwidth': 2,
                    'bordercolor': "gray",
                    'steps': [
                        {'range': red_range, 'color': red_color},
                        {'range': green_range, 'color': green_color},
                    ],
                }
            ))

            # Обновите макет графика
            fig.update_layout(
                autosize=False,
                margin=dict(l=20, r=20, b=20, t=40),
            )

            # Покажите график
            st.plotly_chart(fig)
        else:
            st.write("Erreur : Les données d'entrée sont vides ou n'ont pas deux dimensions.")

#
st.markdown('<div id="section_option2"></div>', unsafe_allow_html=True)

option1 = st.sidebar.checkbox("Facteurs d'influence globaux")
option2 = st.sidebar.checkbox("Facteurs d'influence individuels")
option3 = st.sidebar.checkbox("Comparaison avec les autres clients")

# Создание виджета Spinner
spinner_text = st.empty()
spinner = st.spinner()

# Проверка состояния чекбокса option1
if option1:
    st.title("Facteurs d'influence globaux")
    explainer_shap = shap.TreeExplainer(model)
    df_test_indexed = df_test.set_index('SK_ID_CURR')

    # Проверка, что shap_values не является None
    shap_values = compute_shap_values(explainer_shap, df_test_indexed)
    if shap_values is not None:
        num_columns = st.slider("Sélectionnez le nombre de colonnes", min_value=1, max_value=80, value=10)

        # Запустите Spinner перед выполнением долгой операции
        with spinner:
            # Обновление текста Spinner
            #spinner_text.text("Calcul en cours...")


            st_shap(shap.summary_plot(shap_values[1], df_test_indexed, show=False, max_display=num_columns))
        spinner_text.text("")

    else:
        st.write("Erreur : Les valeurs SHAP ne sont pas disponibles.")
# Проверка опции 2
if option2:
    st.markdown("""
        <script>
            var element = document.getElementById("section_option2");
            element.scrollIntoView({behavior: "smooth"});
        </script>
    """, unsafe_allow_html=True)


if option2:

    st.title("Facteurs d'influence individuels")
    explainer_shap = shap.TreeExplainer(model)
    df_test_indexed = df_test.set_index('SK_ID_CURR')
    idx_option2 = df_test_indexed.index.get_loc(option)

    # Проверка, что shap_values не является None
    shap_values_option2 = compute_shap_values(explainer_shap, df_test_indexed)
    if shap_values_option2 is not None:
        # Add a unique key to the slider widget
        num_columns_2 = st.slider("Sélectionnez le nombre de colonnes", min_value=1, max_value=80, value=10, key="slider_option2")
        # Запустите Spinner перед выполнением долгой операции
        with st.spinner("Calcul en cours..."):
            # Обновление текста Spinner
            #spinner_text.text("Calcul en cours...")

            st.text("")
            left_margin, _, right_margin = st.columns([1, 5, 20])
            left_margin.text("")
            st_shap(shap.plots._waterfall.waterfall_legacy(explainer_shap.expected_value[0], shap_values_option2[0][idx_option2, :], df_test_indexed.iloc[idx_option2, :],  max_display=num_columns_2))
            right_margin.text("")


    else:
        st.write("Erreur : Les valeurs SHAP ne sont pas disponibles.")


# Check Option 3 is Selected

if option3:
    st.markdown('<a href="#section_option2">Aller aux facteurs d\'influence individuels</a>', unsafe_allow_html=True)
    st.title("Analyse comparée avec les autres clients")

    # Значения Shapley для конкретного клиента
    client_shap_values = shap_values_option2[0][idx_option2]

    # Выберите, например, 10 лучших функций
    top_features_indices = np.abs(shap_values_option2[0][idx_option2]).argsort()[-10:][::-1]
    top_features_indices = top_features_indices + 1
    top_features_names = df_test.columns[top_features_indices]

    # Данные для выбранных функций для всех клиентов
    data_for_top_features = df_test[top_features_names]

    import seaborn as sns

    # Выбор типа графика
    # Выбор типа графика
    plot_type = st.radio("Choisir le type de graphique :", ("Histogramme", "Boxplot"))

    # Вычислите количество строк и столбцов в фигуре
    num_rows = 5
    num_columns = 2  # Всегда 2 графика в каждой строке

    # Создайте фигуру и ось
    fig, axs = plt.subplots(num_rows, num_columns, figsize=(15, 20))

    # Переменная для отслеживания текущей позиции в фигуре
    current_row, current_column = 0, 0

    for i, feature_name in enumerate(top_features_names):
        if plot_type == "Histogramme":
            axs[current_row, current_column].hist(data_for_top_features[feature_name], bins=30, color='skyblue', label='All Clients', density=True)
            axs[current_row, current_column].axvline(x=data_for_top_features.loc[idx_option2, feature_name], color='red', linestyle='dashed', linewidth=2, label='Selected Client')
            axs[current_row, current_column].set_title(f'Distribution of {feature_name}\nfor All Clients vs Selected Client')
            axs[current_row, current_column].set_xlabel(feature_name)
            axs[current_row, current_column].set_ylabel('Density')
            axs[current_row, current_column].legend()

        elif plot_type == "Boxplot":
            # Постройте боксплот для каждой функции
            sns.boxplot(x=data_for_top_features[feature_name], color='lightblue', ax=axs[current_row, current_column])

            # Укажите место выбранного клиента на каждом боксплоте
            axs[current_row, current_column].scatter(x=[0], y=[data_for_top_features.loc[idx_option2, feature_name]], color='red', marker='o', label='Selected Client')

            # Настройте оси и метки
            axs[current_row, current_column].set_title(f'Box Plot of {feature_name}\nfor All Clients vs Selected Client')
            axs[current_row, current_column].set_xlabel(feature_name)
            axs[current_row, current_column].set_ylabel('Value')
            axs[current_row, current_column].legend()

            # Установите одинаковый масштаб осей для всех боксплотов
            axs[current_row, current_column].set_ylim(axs[0, 0].get_ylim())

        # Перейдите к следующему столбцу
        current_column += 1
        if current_column == num_columns:
            current_row += 1
            current_column = 0

    # Убедитесь, что все подписи осей видны
    plt.tight_layout()

    # Отобразите график
    st.pyplot(fig, clear_figure=True)







Overwriting app.py


In [194]:
!wget -q -O - ipv4.icanhazip.com


34.80.132.48


In [207]:
!streamlit run app.py & npx localtunnel --port 8501

[..................] / fetchMetadata: sill resolveWithNewModule localtunnel@2.0


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.80.132.48:8501

npx: installed 22 in 2.029s
your url is: https://dirty-garlics-talk.loca.lt
/usr/local/lib/python3.10/dist-packages/shap/explainers/_tree.py:426: UserWarning:

LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray

  Stopping...
^C
